You can create your own search space in AutoML.Net. There're two ways to define a search space: via scratch and via attribution. Via scratch gives you more flexibility while via attribution is more readable. And both ways are equivalant.

In this notebook, we will go through a series of topics on search space
- available options in search space
- create search space from scratch && it's equivalant way via attribution api.
- default search space for mlnet trainers.

In [ ]:
// install dependencies and import using statement
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"

// make sure you are using Microsoft.ML.AutoML later than 0.20.0.
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22356.1 Microsoft.ML.AutoML, 0.20.0-preview.22356.1 Plotly.NET.CSharp, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Plotly.NET.Interactive.dll`

Loading extensions from `Microsoft.ML.AutoML.Interactive.dll`

In [ ]:
// Import usings.
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Microsoft.ML.SearchSpace;
using Newtonsoft.Json;

# Available options in search space.
AutoML.Net search space supports multiple options which should be enough to cover most of usage cases. In summary, it supports
- numeric option - an option that is numeric type, like float, double, int...
- choice option - an option that is descrete, like string or boolean.
- nested option - an option that itself is also a search space.

Underlying, a search space is no more than a json object, where key is option name and value is its value as another json object. This is also how search space supports nested search space. In general, there's no strong limitation on the type of option as long as it can be saved as json, but in practice, it's better to use primitive type since it's well tested.

Once after you create a search space, a n-dimension linear space will be associated with that search space where `n` depends on the # of options and dimension of that option. During hpo, tuner will sample on that n-dimension linear space instead of original options. This feature makes option being transparent to tuner and greatly simplify the implementation of tuner.

# create search space from scratch
The following code shows how to create a search space from scratch and print it out as a json string.

In [ ]:
using Microsoft.ML.SearchSpace.Option;

var searchSpace = new SearchSpace();

// numeric options
searchSpace["IntOption"] = new UniformIntOption(-10, 10, false, 0);
searchSpace["SingleOption"] = new UniformSingleOption(1, 10, true, 1);
searchSpace["DoubleOption"] = new UniformDoubleOption(-10, 10, false, 0);

// choice options
searchSpace["BoolOption"] = new ChoiceOption(true, false);
searchSpace["StrOption"] = new ChoiceOption("a", "b", "c");

// nest options
var nestedSearchSpace = new SearchSpace();
nestedSearchSpace["IntOption"] = new UniformIntOption(-10, 10, false, 0);
searchSpace["Nest"] = nestedSearchSpace;

// check out search space's dimension
Console.WriteLine("search space dimension: " + searchSpace.FeatureSpaceDim);
// pretty print search space
JsonConvert.SerializeObject(searchSpace, Formatting.Indented)

search space dimension: 6


{
  "IntOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.5
    ]
  },
  "SingleOption": {
    "Min": 1.0,
    "Max": 10.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "DoubleOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.5
    ]
  },
  "BoolOption": {
    "Choices": [
      false,
      true
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      2
    ],
    "Default": [
      0.0
    ]
  },
  "StrOption": {
    "Choices": [
      "a",
      "b",
      "c"
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      3
    ],
    "Default": [
      0.0
    ]
  },
  "Nest": {
    "IntOption": {
      "Min": -10.0,
      "Max": 10.0,
      "LogBase": false,
      "FeatureSpaceDim": 1,
      "Step": [
        null
      ],


# create search space from attribution
AutoML allows you to use attribution on property to avoid creating search space from scratch. The following code shows how to create an identical search space from above except using attribution API

In [ ]:
public class MyParameter
{
    [Range((int)-10, 10, 0, false)]
    public int IntOption {get; set;}

    [Range(1f, 10f, 1f, true)]
    public float SingleOption {get; set;}

    [Range(-10, 10, false)]
    public double DoubleOption {get; set;}

    [BooleanChoice]
    public bool BoolOption {get; set;}

    [Choice("a", "b", "c")]
    public string StrOption {get; set;}

    [NestOption]
    public NestParameter Nest {get; set;}
}

public class NestParameter
{
    [Range((int)-10, 10, 0, false)]
    public int IntOption {get; set;}
}

var searchSpace = new SearchSpace<MyParameter>();

// check out search space's dimension
Console.WriteLine("search space dimension: " + searchSpace.FeatureSpaceDim);
// pretty print search space
JsonConvert.SerializeObject(searchSpace, Formatting.Indented)

search space dimension: 6


{
  "IntOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.5
    ]
  },
  "SingleOption": {
    "Min": 1.0,
    "Max": 10.0,
    "LogBase": true,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "DoubleOption": {
    "Min": -10.0,
    "Max": 10.0,
    "LogBase": false,
    "FeatureSpaceDim": 1,
    "Step": [
      null
    ],
    "Default": [
      0.0
    ]
  },
  "BoolOption": {
    "Choices": [
      false,
      true
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      2
    ],
    "Default": [
      0.0
    ]
  },
  "StrOption": {
    "Choices": [
      "a",
      "b",
      "c"
    ],
    "FeatureSpaceDim": 1,
    "Step": [
      3
    ],
    "Default": [
      0.0
    ]
  },
  "Nest": {
    "IntOption": {
      "Min": -10.0,
      "Max": 10.0,
      "LogBase": false,
      "FeatureSpaceDim": 1,
      "Step": [
        null
      ],


# Sampling from search space
In HPO, what tuner does is basically sampling from search space, and pass the sampling result, a.k.a `parameter`, to trial runner. The way of how parameter sampled is what make tuners different from each other and is critial to the final optimizing result. The common tunning algorithems are random search, grid search, smac, eci-cfo and many others.

The following example shows how to sample from the given search space using random search, which sampling from the linear space and mapping it back to the original options.

In [ ]:
var rnd = new Random();
SearchSpace searchSpace = new SearchSpace<MyParameter>();

foreach(var i in Enumerable.Range(0, 3))
{
    var dim = searchSpace.FeatureSpaceDim;
    var featureVector = Enumerable.Range(0, dim).Select(x => rnd.NextDouble()).ToArray();
    var parameter = searchSpace.SampleFromFeatureSpace(featureVector);

    // use Parameter.AsType to map parameter to a concrete type.
    var myParameter = parameter.AsType<MyParameter>();
    var json = JsonConvert.SerializeObject(myParameter);

    Console.WriteLine($"{i} - {json}");

    // equivalant to
    var intOption = parameter["IntOption"].AsType<int>();
    var singleOption = parameter["SingleOption"].AsType<float>();
    var doubleOption = parameter["DoubleOption"].AsType<double>();
    var strOption = parameter["StrOption"].AsType<string>();
    var nest = parameter["Nest"].AsType<NestParameter>();

    myParameter = new MyParameter()
    {
        IntOption = intOption,
        SingleOption = singleOption,
        DoubleOption = doubleOption,
        StrOption = strOption,
        Nest = nest,
    };
    json = JsonConvert.SerializeObject(myParameter);

    Console.WriteLine($"{i} - {json}");
}

0 - {"IntOption":9,"SingleOption":7.392224,"DoubleOption":0.0,"BoolOption":true,"StrOption":"b","Nest":{"IntOption":3}}
0 - {"IntOption":9,"SingleOption":7.392224,"DoubleOption":0.0,"BoolOption":false,"StrOption":"b","Nest":{"IntOption":3}}
1 - {"IntOption":-6,"SingleOption":3.4464028,"DoubleOption":-7.0,"BoolOption":false,"StrOption":"a","Nest":{"IntOption":-5}}
1 - {"IntOption":-6,"SingleOption":3.4464028,"DoubleOption":-7.0,"BoolOption":false,"StrOption":"a","Nest":{"IntOption":-5}}
2 - {"IntOption":-9,"SingleOption":8.262646,"DoubleOption":-7.0,"BoolOption":true,"StrOption":"c","Nest":{"IntOption":2}}
2 - {"IntOption":-9,"SingleOption":8.262646,"DoubleOption":-7.0,"BoolOption":false,"StrOption":"c","Nest":{"IntOption":2}}


# Default search space for ml.net trainer
AutoML.Net comes with a series of default search space for most of ml.net trainers. You can check it under `Microsoft.ML.AutoML.CodeGen` namespace. The following code shows the default search space for LightGbm.

In [ ]:
using Microsoft.ML.AutoML.CodeGen;
var lgbmSearchSpace = new SearchSpace<LgbmOption>();
JsonConvert.SerializeObject(lgbmSearchSpace, Formatting.Indented)

# Continue learning
> E2E-Forecasting using SSA with Luna Dataset (create `SweepableEstimator` with custom search space)